In [8]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from torch.utils.data import Dataset
import pickle
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import torchsample as ts
from torchsample.transforms import RandomRotate
%matplotlib inline

## DATASET

In [9]:
class ImageDataset(Dataset):
    
    def __init__(self, csv_files,transform):
        super().__init__()
        self.csv_files=csv_files
        with open(csv_files[0], 'rb') as f:
            X = np.array(pickle.load(f),dtype=np.float32)
        
        if csv_files[1] is not None:
            with open(csv_files[1], 'rb') as g:
                y = np.array(pickle.load(g))
            le=LabelEncoder()
            y_label=le.fit_transform(y)
            self.y=y_label
            self.le =le 
            
        self.transform=transform
        
        self.X= X.reshape(-1,1,28,28)
#         self.X= X.reshape(-1,1,28,28)
#         print(np.mean(np.mean(self.X,axis=0)))
              
                
    def get_class_count(self):
        return len(self.le.classes_)
    
    def get_input_dim(self):
        return self.X.shape[1]
    
    def get_class_labels(self,y_label):
        le=self.le
        return le.inverse_transform(y_label)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
         z=self.X[idx]
         #print(z.shape)
         z=np.reshape(z, (28,28))
         img_name = Image.fromarray(z)
#         plt.imshow(img_name)
         sample=self.X[idx]
        
         if self.transform is not None:
            sample = self.transform(img_name)
#             #plt.show()
            sample= sample.reshape(1,28,28)         
            return sample,self.y[idx]
         
         return sample

In [10]:
valid_size=0.3
shuffle=True
valid_transform = transforms.Compose([transforms.ToTensor(),  transforms.Normalize(mean=[82.34472], std=[255]),])

augment_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomAffine(30), transforms.ToTensor(),  transforms.Normalize(mean=[82.34472], std=[255]),])
train_transform = transforms.Compose([ transforms.ToTensor(),  transforms.Normalize(mean=[82.34472], std=[255]),])

train_dataset = ImageDataset(['train_image.pkl','train_label.pkl'],train_transform)
valid_dataset = ImageDataset(['train_image.pkl','train_label.pkl'],valid_transform)
augment_dataset = ImageDataset(['train_image.pkl','train_label.pkl'],augment_transform)
#test_dataset = ImageDataset(['test_image.pkl', None],transforms.ToTensor())
batch_size=512
num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
if shuffle:
        np.random.seed(42)
        np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)
augment_loader = torch.utils.data.DataLoader(augment_dataset, batch_size=batch_size, sampler=valid_sampler)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))

In [11]:
import torch.nn as nn

n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = 1000

In [12]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.batch1=nn.BatchNorm2d(32)
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.batch2=nn.BatchNorm2d(64)
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
#         self.relu3 = nn.ReLU()
#         self.batch3=nn.BatchNorm2d(64)
#         # Max pool 3
#         self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(in_features=64 * 4 * 4, out_features=60)
        self.droput = nn.Dropout(p=0.5)
        self.outer = nn.Linear(in_features=60, out_features=4)
        
    def forward(self,x):
        # Convolution 1
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.batch1(out)
        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.batch2(out)
        # Max pool 2 
        out = self.maxpool2(out)
        
#         # Convolution 3 
#         out = self.conv3(out)
#         out = self.relu3(out)
#         out = self.batch3(out)
#         # Max pool 3 
#         out = self.maxpool3(out)

        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.fc2(out)
        out = self.droput(out)
        out = self.outer(out)
        return out


In [13]:
model = Network().cuda(1)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
iter = 0
for epoch in range(num_epochs):
    model.train()
    training_loss_list = []
    testing_loss_list = []
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Load images as tensors with gradient accumulation abilities
        images = images.requires_grad_().cuda(1)
        labels =labels.cuda(1)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        training_loss_list.append(loss.item())
    for i, (images, labels) in enumerate(augment_loader):
    # Load images as tensors with gradient accumulation abilities
        images = images.requires_grad_().cuda(1)
        labels =labels.cuda(1)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        training_loss_list.append(loss.item())
    training_loss = sum(training_loss_list)/len(training_loss_list)    

    model.eval()    
    if(epoch%2==0):
            # Calculate Accuracy         
        correct = 0
        total = 0
            # Iterate through test dataset
        losses = []
        for images, labels in validation_loader:
            # Load images to tensors with gradient accumulation abilities
            images = images.requires_grad_().cuda(1)
            labels = labels.cuda(1)

            # Forward pass only to get logits/output
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            testing_loss_list.append(loss.item())
            # Get predictions from the maximum value
            _, predicted = torch.max(outputs.data, 1)

            # Total number of labels
            total += labels.size(0)

            # Total correct predictions
            correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
        test_loss = sum(testing_loss_list)/len(testing_loss_list)
        torch.save(model.state_dict(),str(epoch)+'.pt')
        print('Epoch: {}. Training_Loss: {}.Testing_Loss:{}. Accuracy: {}'.format(epoch, training_loss,test_loss ,accuracy))

Epoch: 0. Training_Loss: 1.1544707380235195.Testing_Loss:1.0520039558410645. Accuracy: 46
Epoch: 2. Training_Loss: 0.6390603668987751.Testing_Loss:0.6012688398361206. Accuracy: 73
Epoch: 4. Training_Loss: 0.5400331486016512.Testing_Loss:0.5396576702594758. Accuracy: 80
Epoch: 6. Training_Loss: 0.5067481156438589.Testing_Loss:0.4374894917011261. Accuracy: 83
Epoch: 8. Training_Loss: 0.4857434891164303.Testing_Loss:0.5048968195915222. Accuracy: 78
Epoch: 10. Training_Loss: 0.44428982585668564.Testing_Loss:0.45414952039718626. Accuracy: 82
Epoch: 12. Training_Loss: 0.4141511507332325.Testing_Loss:0.4457251846790314. Accuracy: 83
Epoch: 14. Training_Loss: 0.43014665972441435.Testing_Loss:0.4082306087017059. Accuracy: 84
Epoch: 16. Training_Loss: 0.4098108056932688.Testing_Loss:0.3806808054447174. Accuracy: 85
Epoch: 18. Training_Loss: 0.4166389349848032.Testing_Loss:0.3901773691177368. Accuracy: 84
Epoch: 20. Training_Loss: 0.36805864702910185.Testing_Loss:0.3574111878871918. Accuracy: 86


Epoch: 350. Training_Loss: 0.10020910116145387.Testing_Loss:0.12013991922140121. Accuracy: 95
Epoch: 352. Training_Loss: 0.09256020066095516.Testing_Loss:0.1224119320511818. Accuracy: 95
Epoch: 354. Training_Loss: 0.07933640119154006.Testing_Loss:0.17079244256019593. Accuracy: 94
Epoch: 356. Training_Loss: 0.07414029765641317.Testing_Loss:0.08638014495372773. Accuracy: 96
Epoch: 358. Training_Loss: 0.08123918081400916.Testing_Loss:0.0928384706377983. Accuracy: 96
Epoch: 360. Training_Loss: 0.06561074763885699.Testing_Loss:0.11719637662172318. Accuracy: 95
Epoch: 362. Training_Loss: 0.08201865447335877.Testing_Loss:0.08389768227934838. Accuracy: 96
Epoch: 364. Training_Loss: 0.085279377293773.Testing_Loss:0.13081797063350678. Accuracy: 94
Epoch: 366. Training_Loss: 0.10039367887657136.Testing_Loss:0.10463545769453049. Accuracy: 96
Epoch: 368. Training_Loss: 0.09904495539376512.Testing_Loss:0.19530288279056549. Accuracy: 93
Epoch: 370. Training_Loss: 0.08345085402834229.Testing_Loss:0.14

Epoch: 698. Training_Loss: 0.056390134850516915.Testing_Loss:0.07820653617382049. Accuracy: 97
Epoch: 700. Training_Loss: 0.04489869212557096.Testing_Loss:0.04053107351064682. Accuracy: 98
Epoch: 702. Training_Loss: 0.044378623671946116.Testing_Loss:0.05279798731207848. Accuracy: 98
Epoch: 704. Training_Loss: 0.05808200829778798.Testing_Loss:0.06864981725811958. Accuracy: 97
Epoch: 706. Training_Loss: 0.06090794628835283.Testing_Loss:0.03282464407384396. Accuracy: 98
Epoch: 708. Training_Loss: 0.06422174494946375.Testing_Loss:0.0491185799241066. Accuracy: 98
Epoch: 710. Training_Loss: 0.062127521116053686.Testing_Loss:0.06243229061365128. Accuracy: 97
Epoch: 712. Training_Loss: 0.05722687183879316.Testing_Loss:0.07479640990495681. Accuracy: 97
Epoch: 714. Training_Loss: 0.044224382771062665.Testing_Loss:0.03943721614778042. Accuracy: 98
Epoch: 716. Training_Loss: 0.047074524336494505.Testing_Loss:0.07759052515029907. Accuracy: 97
Epoch: 718. Training_Loss: 0.052982556269853376.Testing_

Epoch: 872. Training_Loss: 0.05080971500137821.Testing_Loss:0.07798095718026161. Accuracy: 97
Epoch: 874. Training_Loss: 0.057261342444689944.Testing_Loss:0.0513968039304018. Accuracy: 97
Epoch: 876. Training_Loss: 0.05232733392040245.Testing_Loss:0.044639083370566365. Accuracy: 98
Epoch: 878. Training_Loss: 0.0643799155659508.Testing_Loss:0.0829790249466896. Accuracy: 97
Epoch: 880. Training_Loss: 0.06627903107437305.Testing_Loss:0.13537254929542542. Accuracy: 96
Epoch: 882. Training_Loss: 0.08096447281423025.Testing_Loss:0.08651322722434998. Accuracy: 97
Epoch: 884. Training_Loss: 0.07052337616914883.Testing_Loss:0.06009241789579391. Accuracy: 97
Epoch: 886. Training_Loss: 0.06448745861416683.Testing_Loss:0.046815719828009604. Accuracy: 98
Epoch: 888. Training_Loss: 0.08264955342747271.Testing_Loss:0.08233514577150344. Accuracy: 96
Epoch: 890. Training_Loss: 0.07684090718976222.Testing_Loss:0.1291049227118492. Accuracy: 96
Epoch: 892. Training_Loss: 0.07168211998214247.Testing_Loss:0

In [17]:
import numpy as np
model.load_state_dict(torch.load('550'))
with open('test_image.pkl', 'rb') as f:
    X = np.array(pickle.load(f),dtype=np.float32)

X= X.reshape(-1,1,28,28)    
test_loader=torch.utils.data.DataLoader(X, batch_size=2000)  
for images in test_loader:
    images = images.requires_grad_().cuda(1)
    y = F.softmax(model(images))
    print(y.shape[0])
    _, predicted = torch.max(y.data, 1)
print(predicted)
print(len(predicted))
# access Variable's tensor, copy back to CPU, convert to numpy
arr = predicted.data.cpu().numpy()
arr=np.array(arr,dtype='uint8')
final=train_dataset.get_class_labels(arr)
print(len(final))
# write CSV
np.savetxt('ragja_palakkadavath434.csv', final)

C:\Users\Adarsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


2000
tensor([0, 0, 0,  ..., 3, 3, 3], device='cuda:1')
2000


C:\Users\Adarsh\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


2000
